In [24]:
!pip install wrds
import pandas as pd
import sys
import wrds
from helperfunctions import *
from interests_rate import *
from volatility import vols

In [ ]:
db = wrds.Connection(wrds_username = 'jasminexx')

**Combine options info with prices info**


*   `contracts_ids` returns all options info in that year.

*   `option_prices` returns price info on the traded_date. 



Problem is a contract may be traded in the next calendar year, so we need to check for last two years' optionids.
i.e. `option_prices` for this year contains information of contracts that start in this year and may contains information of contracts that start in last year.

`combined` contains info for an amount of `num_checked` Euro Call contracts each year from 2002 to 2021.


In [ ]:
combined = pd.DataFrame()  # contain all option info from 2002 to 2021
contract_list = []
num_checked = pd.DataFrame()

for year in range(2002, 2022):
  newYearDay = f'{year}-01-01'
  newYearEve = f'{year}-12-31'
  contracts = contract_ids('E',newYearDay,newYearEve)
  contract_list.append(contracts)
  if year != 2002:
    # contracts now contain info of contracts in year and year - 1
    contracts = pd.concat([contract_list.pop(0),contracts], axis = 0, ignore_index = True)
  num_checked = pd.concat([num_checked, contracts.optionid], axis = 0, ignore_index = True)
  all_prices = option_prices(year,contracts)
  combined = combine_option_info(contracts,all_prices,combined)
combined.sort_values(by = ['securityid', 'callput', 'optionid', 'date_traded'], inplace = True, ignore_index = True)
num_checked = len(num_checked[0].unique())

# add a column of moneyness
combined['moneyness'] = combined['underlyings_price'] / combined['strike']

**Add interest rates information**

In [ ]:
data = pd.merge(combined, req_rates_dated, on=['date_traded','days_to_maturity'])

# Company mergers lead to existing contracts being quoted for both securityid's.
data.drop_duplicates(subset = data.columns.difference(['securityid']), ignore_index = True, inplace = True)
# Some repetition of quotes where only difference is strike being slightly changed.
data.drop_duplicates(subset = data.columns.difference(['strike']), ignore_index = True, inplace = True)
# Calculation of days_to_maturity has two answers, typically one day off.
data.drop_duplicates(subset = data.columns.difference(['days_to_maturity', 'rate']), ignore_index = True, inplace = True)
# Some calls are duplicated -- Not sure why.
data.drop_duplicates(subset = data.columns.difference(['optionid']), ignore_index = True, inplace = True)
# Some quotes duplicated across exchanges.
data.drop_duplicates(subset = data.columns.difference(['optionid', 'contract_volume']), ignore_index = True, inplace = True)


# Remove any companys with dividends or stock splits (62% of companies but 1.4% of quotes including puts)
data = data[~data['securityid'].isin(db.get_table('optionm', 'distribution').securityid.unique())]
# There are some outlier contracts which are either accidental trades or the strike/quote multiplier was missed
data = data[(0.5 < data['moneyness']) & (data['moneyness'] < 1.5)]

**Add historical realised volatility information**

In [ ]:
data = pd.merge(data, vols, how = 'left').dropna()

**Summary**

In [ ]:
num_contracts = len(data.optionid.unique())
num_quotes    = len(data.index)
num_calls     = len(data[data['callput'] == 'C'])
num_puts      = len(data.index) - num_calls

data = data[['strike', 'moneyness', 'rate', 'volatility', 'days_to_maturity', 'contract_price']]
data = save_reopen_no_num(data, f'calls_OMrates{len(data.index)}')

if True: print(f"""

This script uses the OptionMetrics interest rates.
These are inconsistent with put-call parity so only
real calls are returned. Another script uses
put-call parity to deduce the rate.

For 2002 to 2021 inclusive, this database has
3,301,638 (to be modified) positive-volume quotes on
1,267,300 (to be modified) distinct Euro call contracts.

This script checked {num_checked:,} Euro-call contracts
and after removing unsuitable quotes, we returned
{num_quotes:,} quotes from {num_contracts:,} contracts;
{100*num_quotes/(3301638):,.1f}% and {100*num_contracts/(1267300):,.1f}% of quotes and contracts respectively.
All are converted to call prices in the output file.

Two-thirds of quotes are unfortunately filtered out
because the Black-Scholes accuracy benchmark performs
exceptionally poorly for quotes with very large volatility
or prices near the ticksize so these are removed as
the error appears systematic (extreme market stress on
particular days rather than the volatility smile itself).

Also, this database is truly enormous: there are
1,096,159,594 (to be modified) total quotes for 2002-2021,
107,988,528 (to be modified) of which are for Euro calls.""")


db.close()